## 탈잉 
### 카테고리 : 외국어

- 총 38 페이지
- 페이지 당 15개 클래스
- likes,link,location,number,price,teacher,title
- 'location'의 경우 타 사이트와의 비교 후 tile과 합칠 예정 `'[' + location + ']' + title`

In [6]:
import scrapy, requests, re 
from scrapy.http import TextResponse

import pandas as pd

In [2]:
# 프로젝트 생성

In [2]:
!scrapy startproject taling_language

New Scrapy project 'taling_language', using template directory 'C:\Users\study\anaconda3\lib\site-packages\scrapy\templates\project', created in:
    C:\Users\study\Desktop\git\Tal_ing\taling_language

You can start your first spider with:
    cd taling_language
    scrapy genspider example example.com


In [4]:
# link

In [7]:
req = requests.get("https://taling.me/Home/Search/?cateMain=5")
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://taling.me/Home/Search/?cateMain=5>

In [6]:
# xpath 찾기 (page 1)

In [8]:
links = response.xpath('//*[@id="top-space"]/div/div/a/@href').extract()
len(links), links

(15,
 ['/Talent/Detail/32461',
  '/Talent/Detail/16353',
  '/Talent/Detail/10668',
  '/Talent/Detail/9522',
  '/Talent/Detail/26419',
  '/Talent/Detail/10408',
  '/Talent/Detail/23850',
  '/Talent/Detail/10570',
  '/Talent/Detail/16620',
  '/Talent/Detail/24263',
  '/Talent/Detail/9947',
  '/Talent/Detail/29581',
  '/Talent/Detail/6024',
  '/Talent/Detail/12119',
  '/Talent/Detail/14395'])

In [9]:
links = list(map(response.urljoin, links))
links

['https://taling.me/Talent/Detail/32461',
 'https://taling.me/Talent/Detail/16353',
 'https://taling.me/Talent/Detail/10668',
 'https://taling.me/Talent/Detail/9522',
 'https://taling.me/Talent/Detail/26419',
 'https://taling.me/Talent/Detail/10408',
 'https://taling.me/Talent/Detail/23850',
 'https://taling.me/Talent/Detail/10570',
 'https://taling.me/Talent/Detail/16620',
 'https://taling.me/Talent/Detail/24263',
 'https://taling.me/Talent/Detail/9947',
 'https://taling.me/Talent/Detail/29581',
 'https://taling.me/Talent/Detail/6024',
 'https://taling.me/Talent/Detail/12119',
 'https://taling.me/Talent/Detail/14395']

In [10]:
# 강좌정보 xpath

In [11]:
link = links[3]
req = requests.get(link)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://taling.me/Talent/Detail/9522>

In [11]:
# title, month(number of time), discount(x), price, category2, teacher, heart, likes, category1, link 
# 탈잉에서 category 분류는 페이지로 구분 

In [14]:
site = '탈잉'
title = '[' + location + ']' + response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/h1/text()')[0].extract()
teacher = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/em/text()')[0].extract()
number = response.xpath('//*[@id="wrap"]/div[2]/aside/div/p/span[2]/span/text()')[0].extract()
s_price = response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/p/span[2]/b/text()')[0].extract()
contentment = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/span/i[2]/text()')[0].extract()
location = response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/div/p[2]/text()')[0].extract().strip()
#soldout = response.xpath('//*[@id="wrap"]/div[2]/aside/div/ul[2]/li[2]/a/text').extract()

site, title, teacher, number, s_price, contentment

('탈잉',
 '[강남 신촌홍대 온라인 Live][3월 모집] 1:1 영어발음교정 : 리뷰가 보장하는 유일무이 전문 발음교정 ',
 'J 튜터',
 '총 5회 10시간',
 '750,000원 /',
 '5.0')

In [13]:
# items.py 

In [14]:
%%writefile taling_language/taling_language/items.py
import scrapy


class TalIngItem(scrapy.Item):
    site = scrapy.Field()
    title = scrapy.Field()
    teacher = scrapy.Field()
    number = scrapy.Field()
    s_price = scrapy.Field()
    contentment = scrapy.Field()
    link = scrapy.Field()
    soldout = scrapy.Field()

Overwriting taling_language/taling_language/items.py


In [15]:
# spider.py

In [16]:
%%writefile taling_language/taling_language/spiders/spider.py
import scrapy
from tal_ing.items import TalIngItem


class TalIngSpider(scrapy.Spider):
    name = "TalingLanguage"
    allow_domain = ["taling.me"]
    pages = range(1, 38+1)
    start_urls = [f"https://taling.me/Home/Search/?page={page}&cateMain=5&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=" for page in pages]
    
    def parse(self, response):
        links = response.xpath('//*[@id="top-space"]/div/div/a/@href').extract()
        links = list(map(response.urljoin, links))
        for link in links:
            #time.sleep(120) 
            yield scrapy.Request(link, callback=self.parse_content)
    
    def parse_content(self, response):
        item = TalIngItem()
        item['site'] = '탈잉'
        item['category1'] = '외국어'
        location = response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/div/p[2]/text()')[0].extract().strip()
        item["title"] = '[' + location + ']' + response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/h1/text()')[0].extract()
        item["teacher"] = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/em/text()')[0].extract()
        try:
            item["number"] = response.xpath('//*[@id="wrap"]/div[2]/aside/div/p/span[2]/span/text()')[0].extract()
        except:
            item["number"] = response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/p/span[2]/span/text()')[0].extract()
        item["price"] = '총 ' + response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/p/span[2]/b/text()')[0].extract().replace(' /', '')
        try:
            item["likes"] = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/span/i[2]/text()')[0].extract()
        except:
            item["likes"] = "(0)"
        soldout = response.xpath('//*[@id="wrap"]/div[2]/aside/div/ul[2]/li[2]/a/text()').extract()
        if soldout not in _:
            pass
        else:
            item["soldout"] = response.xpath('//*[@id="wrap"]/div[2]/aside/div/ul[2]/li[2]/a/text()').extract()
        
        item["link"] = response.url
        #time.sleep(120)
        yield item
    

Writing taling_language/taling_language/spiders/spider.py


In [18]:
pages = range(1, 38+1)
start_urls = [f"https://taling.me/Home/Search/?page={page}&cateMain=5&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=" for page in pages]
req = requests.get(start_urls[0])
response = TextResponse(req.url, body=req.text, encoding="utf-8")

links = response.xpath('//*[@id="top-space"]/div/div/a/@href').extract()
links = list(map(response.urljoin, links))
req = requests.get(links[1])
response = TextResponse(req.url, body=req.text, encoding="utf-8")

site = '탈잉'
location = response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/div/p[2]/text()')[0].extract().strip()
title = '[' + location + ']' + response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/h1/text()')[0].extract()
teacher = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/em/text()')[0].extract()
number = response.xpath('//*[@id="wrap"]/div[2]/aside/div/p/span[2]/span/text()')[0].extract()
price = '총 ' + response.xpath('//*[@id="wrap"]/div[2]/div[3]/div/div[2]/p/span[2]/b/text()')[0].extract().replace(' /', '')
likes = response.xpath('//*[@id="wrap"]/div[2]/div[1]/section[1]/div/div[2]/span/i[2]/text()')[0].extract() 
#soldout= response.xpath('//*[@id="wrap"]/div[2]/aside/div/ul[2]/li[2]/a/text()').extract()
link = response.url

site, title, teacher, number, price, contentment, link

('탈잉',
 '[온라인 Live]★탈잉2020년매출1위★카톡으로 만나는1:1튜터★영어가 바뀌는습관, 내삶이 바뀌는아홉달',
 '방인영 튜터',
 '총 4회 24시간',
 '총 105,600원',
 '5.0',
 'https://taling.me/Talent/Detail/16353')

In [18]:
%pwd

'C:\\Users\\study\\Desktop\\git\\Tal_ing'

In [19]:
%ls taling_language

 Volume in drive C has no label.
 Volume Serial Number is 7DCE-F2C1

 Directory of C:\Users\study\Desktop\git\Tal_ing\tal_ing

2021-03-08  �삤�쟾 06:38    <DIR>          .
2021-03-08  �삤�쟾 06:38    <DIR>          ..
2021-03-08  �삤�쟾 06:38               257 scrapy.cfg
2021-03-08  �삤�쟾 06:38    <DIR>          tal_ing
               1 File(s)            257 bytes
               3 Dir(s)  343,352,250,368 bytes free


In [20]:
%%writefile run.sh
cd C:\Users\study\Desktop\crawling\tal_ing
scrapy crawl TalingLanguage -o taling_language.csv

Overwriting run.sh


In [21]:
!ls

run.sh
tal_ing
tal_ing_hobby.ipynb
tal_ing_language.ipynb
taling_language


In [22]:
%cd taling_language/

C:\Users\study\Desktop\git\Tal_ing\taling_language


In [23]:
!pwd

/c/Users/study/Desktop/git/Tal_ing/taling_language


In [24]:
!/bin/bash run.sh 

The system cannot find the path specified.


In [25]:
!scrapy crawl taling_language -o taling_language.csv -t csv 

C:\Users\study\anaconda3\lib\site-packages\scrapy\commands\__init__.py:131: ScrapyDeprecationWarning: ('The -t command line option is deprecated in favor of specifying the output format within the output URI. See the documentation of the -o and -O options for more information.',)
  feeds = feed_process_params_from_cli(
Traceback (most recent call last):
  File "C:\Users\study\anaconda3\Scripts\scrapy-script.py", line 10, in <module>
    sys.exit(execute())
  File "C:\Users\study\anaconda3\lib\site-packages\scrapy\cmdline.py", line 144, in execute
    cmd.crawler_process = CrawlerProcess(settings)
  File "C:\Users\study\anaconda3\lib\site-packages\scrapy\crawler.py", line 280, in __init__
    super().__init__(settings)
  File "C:\Users\study\anaconda3\lib\site-packages\scrapy\crawler.py", line 152, in __init__
    self.spider_loader = self._get_spider_loader(settings)
  File "C:\Users\study\anaconda3\lib\site-packages\scrapy\crawler.py", line 146, in _get_spider_loader
    return loader

In [26]:
pd.read_csv("tal_ing/taling_language.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'tal_ing/taling_language.csv'

In [19]:
pages = range(1, 38+1)
start_urls = [f"https://taling.me/Home/Search/?page={page}&cateMain=5&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=" for page in pages]
start_urls[:10]

['https://taling.me/Home/Search/?page=1&cateMain=5&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=',
 'https://taling.me/Home/Search/?page=2&cateMain=5&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=',
 'https://taling.me/Home/Search/?page=3&cateMain=5&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=',
 'https://taling.me/Home/Search/?page=4&cateMain=5&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=',
 'https://taling.me/Home/Search/?page=5&cateMain=5&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=',
 'https://taling.me/Home/Search/?page=6&cateMain=5&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=',
 'https://taling.me/Home/Search/?page=7&cateMain=5&cateSub=&region=&orderIdx=&query=&code=&org=&day=&time=&tType=&region=&regionMain=',
 'https://taling.me/Home/Search/?page=8&cateMain